In [14]:
from base_model import *

In [15]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import datasets
# for sampling
import random # use random.choice? and random.sample for interactions

from sklearn.metrics import make_scorer
from sklearn.metrics import *


In [16]:
score_likelihood = make_scorer(gaussian_likelihood)

In [17]:
# create random f
def fpoly(x):
    trans = random.choice(range(2, 5))
    
    def f1(x):
        return x
    
    def f2(x):
        return np.max(x-np.percentile(x, random.choice(range(100))), 0) + x
    
    for _ in range(trans):
        f = random.choice([np.sin, f1, f2,
                          f1, f2, 
                          f1, f2])
        x = x * f(x)
    return x

In [18]:
def eval_pipeline(additional_feats, X, y, verbose=True, scoring=make_scorer(mean_squared_error)):
    #print(additional_feats)    
    pipeline = additional_feats[:]
    pipeline.append(('SGD_regressor', SGDRegressor(loss='squared_loss', penalty='elasticnet')))
    model = Pipeline(pipeline[:])

    # split data into 10 folds
    kfold = KFold(n_splits=10, shuffle=True)
    results = cross_val_score(model, X, y, cv=kfold, 
                              scoring = scoring)
    if verbose:
        print("Result: {}".format(results.mean()))
    return results.mean()

In [19]:
n_samples = 1000
n_outliers = 50
X, y = datasets.make_regression(n_samples=n_samples, n_features=100,
                                      n_informative=100, noise=10,
                                      random_state=0, n_targets=1)

# sample X
cols = random.sample(range(X.shape[1]), 25)
y_poly = y
for col in cols:
    y_poly = fpoly(X[:, col]) + y
y_poly = ((y_poly)/np.mean(y_poly) + 
          np.random.normal(scale=np.std(y), size=y.shape[0]))

In [20]:
# take a random sample
eval_pipeline([], X, y)

Result: 119.85059874502849


119.85059874502849

In [21]:
# take a random sample
eval_pipeline([], X, y_poly)

Result: 384588.898472848


384588.898472848

In [22]:
#eval_pipeline([], X, y_poly, scoring=score_likelihood)

In [23]:
#### do some iterations

In [24]:
bmars = BMARS(X)

In [25]:
bmars.construct_pipeline(False)

[('union',
  FeatureUnion(n_jobs=1, transformer_list=[('base model', BaseModel())],
         transformer_weights=None))]

In [26]:
# evaluate...
eval_pipeline(bmars.construct_pipeline(False), X, y)

Result: 120.35791168051173


120.35791168051173

In [27]:
# one iteration of mh
bk, dk, ck = output_prob_state(0)
u = np.random.uniform()
action = output_action(u, bk, dk, ck)

In [28]:
l = 20.62195407652395

In [29]:
basis = bmars.perform_action(action)

In [30]:
proposed_model = BMARS(**bmars.export())

In [31]:
output = bmars_sample_basis(X, list(basis))
output

{'basis': [65, 99], 'knot': array([ 0.18001937, -1.74177404]), 'sign': [1, -1]}

In [32]:
if action == 'birth':
    output = bmars_sample_basis(X, list(basis), {'signs':[-1, 1]})



In [33]:
# do other things for birth
proposed_model.add_basis(**output)

In [36]:
# evaluate...
eval_pipeline( bmars.construct_pipeline(False), X, y)

Result: 120.4201376707828


120.42013767078279

In [37]:
proposed_model.construct_pipeline(False)

[('union', FeatureUnion(n_jobs=1,
         transformer_list=[('base model', BaseModel()), ('B_6599', Hinge(indices=array([65, 99]), knots=array([-0.583  ,  0.00691]),
     signs=array([-1, -1])))],
         transformer_weights=None))]

In [38]:
a = Hinge(indices=[12, 25], knots=np.array([-1.8652 , -1.07113]), signs=[1, -1])

In [39]:
a.transform(X).reshape(-1, 1).shape

(1000, 1)

In [41]:
# evaluate...
eval_pipeline( proposed_model.construct_pipeline(False), X, y)

Result: 146.99192548242112


146.99192548242112

In [42]:
interaction = 2

In [43]:
accept_proposal_ratio(X, y, l, interaction, bmars, proposed_model, mode='birth')

20200000000.0

In [44]:
accept_proposal_ratio(X, y, l, interaction, proposed_model, bmars, mode='death')

4.9504950495049507e-11

In [45]:
acceptance_proba(X, y, l, interaction, bmars, proposed_model, mode='birth')

ValueError: bad input shape ()